In [1]:
import warnings
warnings.filterwarnings('ignore')


In [2]:
from pathlib import Path
import requests
from io import BytesIO
from zipfile import ZipFile, BadZipFile

import numpy as np
import pandas as pd
import pandas_datareader.data as web
from sklearn.datasets import fetch_openml

pd.set_option('display.expand_frame_repr', False)

In [3]:
DATA_STORE = Path('assets.h5')

In [4]:
import pandas as pd
import gdown
import os

# File path
file_path = 'wiki_prices.csv'

# Download the file from Google Drive only if it doesn't exist
if not os.path.exists(file_path):
    gdown.download('https://drive.google.com/uc?export=download&id=1RTEvR3S_jNZNIn_WldbCKfsDb2pl3KZf', file_path, quiet=False)
else:
    print(f"File '{file_path}' already exists. Skipping download.")

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path,
                 parse_dates=['date'],
                 index_col=['date', 'ticker'],
                 infer_datetime_format=True).sort_index()

# Now you can work with the DataFrame 'df'


print(df.info())
with pd.HDFStore(DATA_STORE) as store:
    store.put('quandl/wiki/prices', df)

File 'wiki_prices.csv' already exists. Skipping download.
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 15389314 entries, (Timestamp('1962-01-02 00:00:00'), 'ARNC') to (Timestamp('2018-03-27 00:00:00'), 'ZUMZ')
Data columns (total 12 columns):
 #   Column       Dtype  
---  ------       -----  
 0   open         float64
 1   high         float64
 2   low          float64
 3   close        float64
 4   volume       float64
 5   ex-dividend  float64
 6   split_ratio  float64
 7   adj_open     float64
 8   adj_high     float64
 9   adj_low      float64
 10  adj_close    float64
 11  adj_volume   float64
dtypes: float64(12)
memory usage: 1.4+ GB
None
